In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION
# =============================================================================
TARGETS = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']
TEMPORAL_FEATURES = ['hour', 'is_day', 'hour_sin', 'hour_cos', 'dow', 'dow_sin',
                     'dow_cos', 'is_holiday', 'is_weekend', 'lockdown_code']
LAGS = [6, 12, 24]

DATE_COL = 'date'  # Adjust to your actual datetime column name

# Cross-validation settings
N_SPLITS = 3
TEST_SIZE_RATIO = 0.2

FEATURE_STRATEGY = 'temporal_only'

# SARIMAX parameters - adjust based on your data
# For hourly data with daily patterns: s=24
# For daily data with weekly patterns: s=7
SARIMAX_ORDER = (2, 1, 2)  # Increase p,q to compensate
SARIMAX_SEASONAL_ORDER = (0, 0, 0, 0)  # No seasonality
MAX_ITER = 100

# =============================================================================
# LOAD AND PREPARE DATA
# =============================================================================
print("="*70)
print("LOADING AND PREPARING DATA")
print("="*70)

train_df = pd.read_csv("../data/train_features.csv")
test_df = pd.read_csv("../data/test_features_to_predict.csv")

print(f"\n✓ Loaded train: {train_df.shape}")
print(f"✓ Loaded test: {test_df.shape}")

# Convert date column if it exists
if DATE_COL in train_df.columns:
    train_df[DATE_COL] = pd.to_datetime(train_df[DATE_COL])
    train_df = train_df.sort_values(DATE_COL).reset_index(drop=True)

if DATE_COL in test_df.columns:
    test_df[DATE_COL] = pd.to_datetime(test_df[DATE_COL])
    test_df = test_df.sort_values(DATE_COL).reset_index(drop=True)

# Identify all lag/roll columns
all_cols = train_df.columns.tolist()
lag_roll_cols = [c for c in all_cols if 'lag_' in c or 'roll_' in c]
feature_cols = lag_roll_cols + TEMPORAL_FEATURES

print(f"\nFeature breakdown:")
print(f"  - Lag/Roll features in data: {len(lag_roll_cols)}")
print(f"  - Temporal features: {len(TEMPORAL_FEATURES)}")

# =============================================================================
# SMART FEATURE SELECTION FOR SARIMAX
# =============================================================================
def get_cross_pollutant_features(target_col):
    """
    Get rolling means of OTHER pollutants (not the target being predicted).
    Only keep rolling means, not lags (SARIMAX handles lags internally).
    """
    target_name = target_col.replace('valeur_', '')

    # All pollutants except current target
    other_pollutants = [p for p in ['NO2', 'CO', 'O3', 'PM10', 'PM25']
                        if p != target_name]

    # Only rolling means (more stable than lags, captures smoothed relationships)
    cross_features = []
    for pollutant in other_pollutants:
        cross_features.extend([
            f'valeur_{pollutant}_roll_mean_6',
            f'valeur_{pollutant}_roll_mean_24'
        ])

    return cross_features

def prepare_features_for_target(df, target_col, strategy='optimized'):
    """
    Prepare optimized feature set for SARIMAX based on strategy.

    Strategies:
    - 'temporal_only': Just temporal features (fastest, cleanest)
    - 'optimized': Temporal + current weather + cross-pollutant rolling means
    - 'all': All lag/roll features (not recommended for SARIMAX)
    """
    if strategy == 'temporal_only':
        features = TEMPORAL_FEATURES.copy()

    elif strategy == 'optimized':
        features = TEMPORAL_FEATURES.copy()

        # Add current weather if available
        weather_available = [w for w in WEATHER_CURRENT if w in df.columns]
        features.extend(weather_available)

        # Add cross-pollutant rolling means
        cross_features = get_cross_pollutant_features(target_col)
        cross_available = [f for f in cross_features if f in df.columns]
        features.extend(cross_available)

    elif strategy == 'all':
        # Use all lag/roll features (original approach)
        features = lag_roll_cols + TEMPORAL_FEATURES

    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    # Filter to only available columns
    available_features = [f for f in features if f in df.columns]

    return available_features

# Display feature strategy
print(f"\n{'='*70}")
print(f"FEATURE STRATEGY: {FEATURE_STRATEGY}")
print(f"{'='*70}")

# Show what features will be used for first target (as example)
sample_features = prepare_features_for_target(train_df, TARGETS[0], FEATURE_STRATEGY)
print(f"\nExample features for {TARGETS[0]}: {len(sample_features)} features")
print(f"  - Temporal: {len([f for f in sample_features if f in TEMPORAL_FEATURES])}")
print(f"  - Weather: {len([f for f in sample_features if f in WEATHER_CURRENT])}")
print(f"  - Cross-pollutant: {len([f for f in sample_features if 'roll_mean' in f])}")
print(f"\nFeature list:")
for f in sample_features[:20]:  # Show first 20
    print(f"    {f}")
if len(sample_features) > 20:
    print(f"    ... and {len(sample_features) - 20} more")

# Clean training data - use selected features + targets
feature_cols = prepare_features_for_target(train_df, TARGETS[0], FEATURE_STRATEGY)
train_clean = train_df[feature_cols + TARGETS].dropna().reset_index(drop=True)
print(f"\n✓ Train samples after removing NaNs: {len(train_clean)}")

# =============================================================================
# TIME SERIES CROSS-VALIDATION SPLITTER
# =============================================================================
def time_series_cv_split(df, n_splits=5, test_ratio=0.2):
    """
    Creates time series cross-validation splits using expanding window.
    """
    n = len(df)
    test_size = int(n * test_ratio)
    min_train_size = int(n * 0.5)

    splits = []
    for i in range(n_splits):
        val_end = n - (n_splits - i - 1) * (test_size // 2)
        val_start = val_end - test_size
        train_end = val_start

        if train_end < min_train_size:
            continue

        train_idx = df.index[:train_end]
        val_idx = df.index[val_start:val_end]
        splits.append((train_idx, val_idx))

    print(f"\n✓ Created {len(splits)} CV splits:")
    for i, (train_idx, val_idx) in enumerate(splits, 1):
        print(f"  Fold {i}: Train={len(train_idx)}, Val={len(val_idx)}")

    return splits

# =============================================================================
# PREPARE EXOGENOUS FEATURES
# =============================================================================
def prepare_exog_features(df, target_col, strategy='optimized'):
    """
    Extract exogenous features from dataframe based on strategy.
    Returns None if no features, otherwise returns feature matrix.
    """
    features = prepare_features_for_target(df, target_col, strategy)

    if len(features) == 0:
        return None

    # Get only the feature columns that exist
    available_cols = [c for c in features if c in df.columns]

    if len(available_cols) == 0:
        return None

    exog_df = df[available_cols].copy()

    # Handle any remaining NaNs
    if exog_df.isna().any().any():
        exog_df = exog_df.fillna(method='ffill').fillna(method='bfill')

    return exog_df

# =============================================================================
# TRAIN SARIMAX WITH CROSS-VALIDATION
# =============================================================================
def train_sarimax_with_cv(df, target_col, n_splits=5):
    """
    Train SARIMAX model using time series cross-validation.
    Features are automatically selected based on FEATURE_STRATEGY.
    """
    print(f"\n{'='*70}")
    print(f"TRAINING SARIMAX FOR {target_col}")
    print(f"{'='*70}")

    splits = time_series_cv_split(df, n_splits=n_splits, test_ratio=TEST_SIZE_RATIO)
    cv_scores = []

    # Cross-validation loop
    for fold, (train_idx, val_idx) in enumerate(splits, 1):
        print(f"\n--- Fold {fold}/{len(splits)} ---")

        train_data = df.loc[train_idx]
        val_data = df.loc[val_idx]

        # Prepare target and exogenous features
        y_train = train_data[target_col]
        y_val = val_data[target_col]

        exog_train = prepare_exog_features(train_data, target_col, FEATURE_STRATEGY)
        exog_val = prepare_exog_features(val_data, target_col, FEATURE_STRATEGY)

        try:
            # Fit SARIMAX
            model = SARIMAX(
                endog=y_train,
                exog=exog_train,
                order=SARIMAX_ORDER,
                seasonal_order=SARIMAX_SEASONAL_ORDER,
                enforce_stationarity=False,
                enforce_invertibility=False
            )

            model_fit = model.fit(disp=False, maxiter=MAX_ITER, method='lbfgs')

            # Forecast
            predictions = model_fit.forecast(steps=len(val_idx), exog=exog_val)
            predictions = np.maximum(predictions, 0)  # Non-negative

            # Metrics
            rmse = np.sqrt(mean_squared_error(y_val, predictions))
            mae = mean_absolute_error(y_val, predictions)
            r2 = r2_score(y_val, predictions)

            cv_scores.append({'fold': fold, 'rmse': rmse, 'mae': mae, 'r2': r2})
            print(f"  RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

        except Exception as e:
            print(f"  ✗ Error: {str(e)}")
            continue

    # Display CV results
    if cv_scores:
        cv_df = pd.DataFrame(cv_scores)
        print(f"\n{'─'*70}")
        print(f"CV Summary for {target_col}:")
        print(cv_df.to_string(index=False))
        print(f"\nMean RMSE: {cv_df['rmse'].mean():.4f} (± {cv_df['rmse'].std():.4f})")
        print(f"Mean MAE:  {cv_df['mae'].mean():.4f} (± {cv_df['mae'].std():.4f})")
        print(f"Mean R²:   {cv_df['r2'].mean():.4f} (± {cv_df['r2'].std():.4f})")

    # Train final model on full data
    print(f"\n🔧 Training final model on full training set...")
    y_full = df[target_col]
    exog_full = prepare_exog_features(df, target_col, FEATURE_STRATEGY)

    final_model = SARIMAX(
        endog=y_full,
        exog=exog_full,
        order=SARIMAX_ORDER,
        seasonal_order=SARIMAX_SEASONAL_ORDER,
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    final_model_fit = final_model.fit(disp=False, maxiter=MAX_ITER, method='lbfgs')
    print(f"✓ Final model trained")

    return final_model_fit, cv_scores

# =============================================================================
# TRAIN MODELS FOR ALL TARGETS
# =============================================================================
print(f"\n{'='*70}")
print("TRAINING MODELS FOR ALL POLLUTANTS")
print(f"{'='*70}")

models = {}
all_cv_scores = {}

for i, target in enumerate(TARGETS, 1):
    print(f"\n[{i}/{len(TARGETS)}] Processing {target}...")

    try:
        model_fit, cv_scores = train_sarimax_with_cv(
            train_clean,
            target,
            n_splits=N_SPLITS
        )
        models[target] = model_fit
        all_cv_scores[target] = cv_scores
        print(f"✓ {target} model trained successfully")

    except Exception as e:
        print(f"✗ Failed to train {target}: {str(e)}")
        models[target] = None
        all_cv_scores[target] = []

successful_models = sum(1 for m in models.values() if m is not None)
print(f"\n{'='*70}")
print(f"Training Complete: {successful_models}/{len(TARGETS)} models successful")
print(f"{'='*70}")

# =============================================================================
# PREPARE TEST FEATURES - SMART APPROACH
# =============================================================================
print(f"\n{'='*70}")
print("PREPARING TEST DATA")
print(f"{'='*70}")

# Get last 24 rows of training data for feature initialization
last_24_rows = train_df.iloc[-24:].copy()
print(f"\n✓ Using last 24 rows of train for lag/roll feature averaging")

# Build test features row by row
test_features_dict = {}  # Store features per target

for target in TARGETS:
    print(f"\nPreparing test features for {target}...")

    # Get feature list for this target
    feature_list = prepare_features_for_target(test_df, target, FEATURE_STRATEGY)

    test_features_list = []

    for idx in range(len(test_df)):
        test_row = test_df.iloc[idx]

        row_features = []

        for col in feature_list:
            if col in TEMPORAL_FEATURES:
                # Temporal features: use actual test values
                if col in test_row.index:
                    feat_val = float(test_row[col])
                else:
                    feat_val = 0.0

            elif col in WEATHER_CURRENT:
                # Current weather: use actual test values
                if col in test_row.index:
                    feat_val = float(test_row[col])
                else:
                    feat_val = 0.0

            elif col in last_24_rows.columns:
                # Lag/roll features: average from last 24 train rows
                feat_val = float(last_24_rows[col].mean())
            else:
                feat_val = 0.0

            row_features.append(feat_val)

        test_features_list.append(row_features)

    # Create DataFrame for this target's test features
    test_features_df = pd.DataFrame(test_features_list, columns=feature_list)
    test_features_dict[target] = test_features_df

    print(f"  ✓ Shape: {test_features_df.shape}")

# =============================================================================
# MAKE PREDICTIONS ON TEST SET
# =============================================================================
print(f"\n{'='*70}")
print("GENERATING PREDICTIONS FOR TEST SET")
print(f"{'='*70}")

# Initialize predictions dataframe
if DATE_COL in test_df.columns:
    predictions_df = test_df[[DATE_COL]].copy()
else:
    predictions_df = pd.DataFrame(index=test_df.index)

for target in TARGETS:
    print(f"\n🔮 Predicting {target}...")

    if models[target] is None:
        print(f"  ✗ No model available, using zeros")
        predictions_df[target] = 0
        continue

    try:
        # Get the correct test features for this target
        test_features_df = test_features_dict[target]

        # Make predictions using the prepared test features
        n_steps = len(test_features_df)
        preds = models[target].forecast(steps=n_steps, exog=test_features_df)

        # Ensure non-negative predictions
        preds = np.maximum(preds, 0)

        predictions_df[target] = preds
        print(f"  ✓ Predicted {target}")
        print(f"    Range: [{preds.min():.2f}, {preds.max():.2f}]")
        print(f"    Mean: {preds.mean():.2f}")

    except Exception as e:
        print(f"  ✗ Error predicting {target}: {str(e)}")
        predictions_df[target] = 0

# =============================================================================
# SAVE RESULTS
# =============================================================================
print(f"\n{'='*70}")
print("SAVING RESULTS")
print(f"{'='*70}")

# Save submission file
predictions_df.to_csv('submission.csv', index=False)
print(f"\n✓ Submission saved as 'submission.csv'")
print(f"  Shape: {predictions_df.shape}")
print(f"\nFirst few predictions:")
print(predictions_df.head(10))

# Save CV scores summary
cv_summary = []
for target, scores in all_cv_scores.items():
    if scores:
        cv_df = pd.DataFrame(scores)
        cv_summary.append({
            'target': target,
            'mean_rmse': cv_df['rmse'].mean(),
            'std_rmse': cv_df['rmse'].std(),
            'mean_mae': cv_df['mae'].mean(),
            'std_mae': cv_df['mae'].std(),
            'mean_r2': cv_df['r2'].mean(),
            'std_r2': cv_df['r2'].std()
        })

if cv_summary:
    cv_summary_df = pd.DataFrame(cv_summary)
    cv_summary_df.to_csv('cv_scores_summary.csv', index=False)
    print(f"\n✓ CV scores saved as 'cv_scores_summary.csv'")
    print("\nCross-Validation Summary:")
    print(cv_summary_df.to_string(index=False))

print(f"\n{'='*70}")
print("PIPELINE COMPLETE!")
print(f"{'='*70}")

LOADING AND PREPARING DATA

✓ Loaded train: (40991, 213)
✓ Loaded test: (504, 208)

Feature breakdown:
  - Lag/Roll features in data: 194
  - Temporal features: 10

FEATURE STRATEGY: temporal_only

Example features for valeur_NO2: 10 features
  - Temporal: 10
  - Weather: 0
  - Cross-pollutant: 0

Feature list:
    hour
    is_day
    hour_sin
    hour_cos
    dow
    dow_sin
    dow_cos
    is_holiday
    is_weekend
    lockdown_code

✓ Train samples after removing NaNs: 40991

TRAINING MODELS FOR ALL POLLUTANTS

[1/5] Processing valeur_NO2...

TRAINING SARIMAX FOR valeur_NO2

✓ Created 3 CV splits:
  Fold 1: Train=24595, Val=8198
  Fold 2: Train=28694, Val=8198
  Fold 3: Train=32793, Val=8198

--- Fold 1/3 ---
  RMSE: 15.5591 | MAE: 13.0544 | R²: -0.1726

--- Fold 2/3 ---
  RMSE: 14.4749 | MAE: 10.5844 | R²: 0.0061

--- Fold 3/3 ---
  RMSE: 16.6503 | MAE: 14.4529 | R²: -0.4749

──────────────────────────────────────────────────────────────────────
CV Summary for valeur_NO2:
 fold    

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION - NO WEATHER FEATURES
# =============================================================================
TARGETS = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

TEMPORAL_FEATURES = ['hour', 'is_day', 'hour_sin', 'hour_cos', 'dow', 'dow_sin',
                     'dow_cos', 'is_holiday', 'is_weekend', 'lockdown_code']

DATE_COL = 'date'

# Cross-validation settings - optimized for speed
N_SPLITS = 2
TEST_SIZE_RATIO = 0.15

# SARIMAX with STRONGER seasonality to compensate for missing weather
SARIMAX_ORDER = (1, 1, 1)
SARIMAX_SEASONAL_ORDER = (1, 1, 1, 24)  # Full seasonality for daily patterns
MAX_ITER = 75

# =============================================================================
# LOAD AND PREPARE DATA
# =============================================================================
print("="*70)
print("SARIMAX PIPELINE - TEMPORAL + CROSS-POLLUTANT FEATURES")
print("="*70)

train_df = pd.read_csv("../data/train_features.csv")
test_df = pd.read_csv("../data/test_features_to_predict.csv")

print(f"\n✓ Loaded train: {train_df.shape}")
print(f"✓ Loaded test: {test_df.shape}")

# Convert and sort by date
if DATE_COL in train_df.columns:
    train_df[DATE_COL] = pd.to_datetime(train_df[DATE_COL])
    train_df = train_df.sort_values(DATE_COL).reset_index(drop=True)

if DATE_COL in test_df.columns:
    test_df[DATE_COL] = pd.to_datetime(test_df[DATE_COL])
    test_df = test_df.sort_values(DATE_COL).reset_index(drop=True)

# =============================================================================
# FEATURE SELECTION - TEMPORAL + CROSS-POLLUTANT
# =============================================================================
def get_cross_pollutant_features(target_col):
    """Get rolling means of OTHER pollutants."""
    target_name = target_col.replace('valeur_', '')
    other_pollutants = [p for p in ['NO2', 'CO', 'O3', 'PM10', 'PM25']
                        if p != target_name]

    cross_features = []
    for pollutant in other_pollutants:
        cross_features.append(f'valeur_{pollutant}_roll_mean_24')

    return cross_features

def prepare_features_for_target(df, target_col):
    """Prepare feature set: temporal + cross-pollutant only."""
    features = TEMPORAL_FEATURES.copy()

    # Add cross-pollutant rolling means
    cross_features = get_cross_pollutant_features(target_col)
    cross_available = [f for f in cross_features if f in df.columns]
    features.extend(cross_available)

    # Filter to available columns
    available_features = [f for f in features if f in df.columns]

    return available_features

# Display features
print(f"\n{'='*70}")
print(f"FEATURE CONFIGURATION")
print(f"{'='*70}")

sample_features = prepare_features_for_target(train_df, TARGETS[0])
print(f"\nFeatures for {TARGETS[0]}: {len(sample_features)} total")
print(f"  - Temporal: {len([f for f in sample_features if f in TEMPORAL_FEATURES])}")
print(f"  - Cross-pollutant: {len([f for f in sample_features if 'roll_mean' in f])}")

# Clean training data
all_needed_cols = set()
for target in TARGETS:
    all_needed_cols.update(prepare_features_for_target(train_df, target))
all_needed_cols.update(TARGETS)

train_clean = train_df[list(all_needed_cols)].dropna().reset_index(drop=True)
print(f"\n✓ Clean train samples: {len(train_clean)}")

# =============================================================================
# TIME SERIES CROSS-VALIDATION
# =============================================================================
def time_series_cv_split(df, n_splits=2, test_ratio=0.15):
    """Expanding window CV splits."""
    n = len(df)
    test_size = int(n * test_ratio)
    min_train_size = int(n * 0.6)

    splits = []
    for i in range(n_splits):
        val_end = n - (n_splits - i - 1) * (test_size // 2)
        val_start = val_end - test_size
        train_end = val_start

        if train_end < min_train_size:
            continue

        train_idx = df.index[:train_end]
        val_idx = df.index[val_start:val_end]
        splits.append((train_idx, val_idx))

    return splits

def prepare_exog_features(df, target_col):
    """Extract exogenous features for target."""
    features = prepare_features_for_target(df, target_col)

    if len(features) == 0:
        return None

    available_cols = [c for c in features if c in df.columns]
    if len(available_cols) == 0:
        return None

    exog_df = df[available_cols].copy()

    # Handle NaNs
    if exog_df.isna().any().any():
        exog_df = exog_df.fillna(method='ffill').fillna(method='bfill')

    return exog_df

# =============================================================================
# TRAIN SARIMAX WITH CV
# =============================================================================
def train_sarimax_with_cv(df, target_col, n_splits=2):
    """Train SARIMAX with time series cross-validation."""
    print(f"\n{'='*70}")
    print(f"TRAINING: {target_col}")
    print(f"{'='*70}")

    splits = time_series_cv_split(df, n_splits=n_splits, test_ratio=TEST_SIZE_RATIO)
    print(f"✓ Created {len(splits)} CV splits")

    cv_scores = []

    for fold, (train_idx, val_idx) in enumerate(splits, 1):
        print(f"\nFold {fold}/{len(splits)} - Train: {len(train_idx)}, Val: {len(val_idx)}", end=" ")

        train_data = df.loc[train_idx]
        val_data = df.loc[val_idx]

        y_train = train_data[target_col]
        y_val = val_data[target_col]

        exog_train = prepare_exog_features(train_data, target_col)
        exog_val = prepare_exog_features(val_data, target_col)

        try:
            model = SARIMAX(
                endog=y_train,
                exog=exog_train,
                order=SARIMAX_ORDER,
                seasonal_order=SARIMAX_SEASONAL_ORDER,
                enforce_stationarity=False,
                enforce_invertibility=False
            )

            model_fit = model.fit(disp=False, maxiter=MAX_ITER, method='lbfgs')

            predictions = model_fit.forecast(steps=len(val_idx), exog=exog_val)
            predictions = np.maximum(predictions, 0)

            rmse = np.sqrt(mean_squared_error(y_val, predictions))
            mae = mean_absolute_error(y_val, predictions)
            r2 = r2_score(y_val, predictions)

            cv_scores.append({'fold': fold, 'rmse': rmse, 'mae': mae, 'r2': r2})
            print(f"→ RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.3f}")

        except Exception as e:
            print(f"→ ✗ Error: {str(e)[:50]}")
            continue

    # Summary
    if cv_scores:
        cv_df = pd.DataFrame(cv_scores)
        print(f"\n📊 CV Summary: RMSE={cv_df['rmse'].mean():.2f}±{cv_df['rmse'].std():.2f}, "
              f"MAE={cv_df['mae'].mean():.2f}±{cv_df['mae'].std():.2f}, "
              f"R²={cv_df['r2'].mean():.3f}±{cv_df['r2'].std():.3f}")

    # Train final model
    print(f"🔧 Training final model on full data...", end=" ")
    y_full = df[target_col]
    exog_full = prepare_exog_features(df, target_col)

    final_model = SARIMAX(
        endog=y_full,
        exog=exog_full,
        order=SARIMAX_ORDER,
        seasonal_order=SARIMAX_SEASONAL_ORDER,
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    final_model_fit = final_model.fit(disp=False, maxiter=MAX_ITER, method='lbfgs')
    print(f"✓ Done")

    return final_model_fit, cv_scores

# =============================================================================
# TRAIN ALL MODELS
# =============================================================================
print(f"\n{'='*70}")
print("TRAINING ALL POLLUTANT MODELS")
print(f"{'='*70}")

models = {}
all_cv_scores = {}

import time
start_time = time.time()

for i, target in enumerate(TARGETS, 1):
    print(f"\n[{i}/{len(TARGETS)}] {target}")

    try:
        model_fit, cv_scores = train_sarimax_with_cv(
            train_clean,
            target,
            n_splits=N_SPLITS
        )
        models[target] = model_fit
        all_cv_scores[target] = cv_scores

    except Exception as e:
        print(f"✗ Failed: {str(e)[:100]}")
        models[target] = None
        all_cv_scores[target] = []

elapsed = time.time() - start_time
successful = sum(1 for m in models.values() if m is not None)
print(f"\n{'='*70}")
print(f"✓ Training Complete: {successful}/{len(TARGETS)} models successful")
print(f"⏱ Total time: {elapsed/60:.1f} minutes")
print(f"{'='*70}")

# =============================================================================
# PREPARE TEST FEATURES
# =============================================================================
print(f"\n{'='*70}")
print("PREPARING TEST FEATURES")
print(f"{'='*70}")

last_24_rows = train_df.iloc[-24:].copy()
test_features_dict = {}

for target in TARGETS:
    feature_list = prepare_features_for_target(test_df, target)
    test_features_list = []

    for idx in range(len(test_df)):
        test_row = test_df.iloc[idx]
        row_features = []

        for col in feature_list:
            if col in TEMPORAL_FEATURES:
                # Use actual test values for temporal features
                feat_val = float(test_row[col]) if col in test_row.index else 0.0
            elif col in last_24_rows.columns:
                # Use average from last 24 hours for cross-pollutant features
                feat_val = float(last_24_rows[col].mean())
            else:
                feat_val = 0.0

            row_features.append(feat_val)

        test_features_list.append(row_features)

    test_features_dict[target] = pd.DataFrame(test_features_list, columns=feature_list)

print(f"✓ Test features prepared for all {len(TARGETS)} targets")

# =============================================================================
# MAKE PREDICTIONS
# =============================================================================
print(f"\n{'='*70}")
print("GENERATING PREDICTIONS")
print(f"{'='*70}")

if DATE_COL in test_df.columns:
    predictions_df = test_df[[DATE_COL]].copy()
else:
    predictions_df = pd.DataFrame(index=test_df.index)

for target in TARGETS:
    if models[target] is None:
        print(f"✗ {target}: No model, using zeros")
        predictions_df[target] = 0
        continue

    try:
        test_features = test_features_dict[target]
        preds = models[target].forecast(steps=len(test_features), exog=test_features)
        preds = np.maximum(preds, 0)

        predictions_df[target] = preds
        print(f"✓ {target}: [{preds.min():.1f}, {preds.max():.1f}], mean={preds.mean():.1f}")

    except Exception as e:
        print(f"✗ {target}: Error - {str(e)[:50]}")
        predictions_df[target] = 0

# =============================================================================
# SAVE RESULTS
# =============================================================================
print(f"\n{'='*70}")
print("SAVING RESULTS")
print(f"{'='*70}")

predictions_df.to_csv('submission.csv', index=False)
print(f"✓ Submission saved: submission.csv ({predictions_df.shape})")

# Save CV scores
cv_summary = []
for target, scores in all_cv_scores.items():
    if scores:
        cv_df = pd.DataFrame(scores)
        cv_summary.append({
            'target': target,
            'mean_rmse': cv_df['rmse'].mean(),
            'std_rmse': cv_df['rmse'].std(),
            'mean_mae': cv_df['mae'].mean(),
            'std_mae': cv_df['mae'].std(),
            'mean_r2': cv_df['r2'].mean(),
            'std_r2': cv_df['r2'].std()
        })

if cv_summary:
    cv_summary_df = pd.DataFrame(cv_summary)
    cv_summary_df.to_csv('cv_scores_summary.csv', index=False)
    print(f"✓ CV scores saved: cv_scores_summary.csv")
    print(f"\n📊 FINAL RESULTS:")
    print(cv_summary_df.to_string(index=False))

print(f"\n{'='*70}")
print("✅ PIPELINE COMPLETE!")
print(f"{'='*70}")
print(f"\n💡 Model Configuration:")
print(f"   SARIMAX Order: {SARIMAX_ORDER} × {SARIMAX_SEASONAL_ORDER}")
print(f"   Features: Temporal + Cross-Pollutant (NO weather)")
print(f"   CV Folds: {N_SPLITS}")
print(f"   Total runtime: {elapsed/60:.1f} minutes")

SARIMAX PIPELINE - TEMPORAL + CROSS-POLLUTANT FEATURES

✓ Loaded train: (40991, 213)
✓ Loaded test: (504, 208)

FEATURE CONFIGURATION

Features for valeur_NO2: 14 total
  - Temporal: 10
  - Cross-pollutant: 4

✓ Clean train samples: 40991

TRAINING ALL POLLUTANT MODELS

[1/5] valeur_NO2

TRAINING: valeur_NO2
✓ Created 2 CV splits

Fold 1/2 - Train: 31769, Val: 6148 → RMSE: 25.76, MAE: 19.57, R²: -1.699

Fold 2/2 - Train: 34843, Val: 6148 → RMSE: 20.98, MAE: 17.35, R²: -2.212

📊 CV Summary: RMSE=23.37±3.38, MAE=18.46±1.57, R²=-1.955±0.363
🔧 Training final model on full data... ✓ Done

[2/5] valeur_CO

TRAINING: valeur_CO
✓ Created 2 CV splits

Fold 1/2 - Train: 31769, Val: 6148 → RMSE: 0.09, MAE: 0.05, R²: 0.279

Fold 2/2 - Train: 34843, Val: 6148 → ✗ Error: Unable to allocate 691. MiB for an array with shap

📊 CV Summary: RMSE=0.09±nan, MAE=0.05±nan, R²=0.279±nan
🔧 Training final model on full data... ✗ Failed: Unable to allocate 813. MiB for an array with shape (51, 51, 40992) and dat